# This is the pytorch version of karpathy's charnn
    reference: https://gist.github.com/raphaelbastide/11ae4bb5e454e5c5239f



In [1]:
import torch
from torch import nn
from torch.autograd import Variable
import torch.optim as optim

In [2]:
# data I/O
data = open('input1.txt', 'r', encoding='utf-8').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print ('data has {} characters, {} unique.'.format( data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 1947 characters, 521 unique.


In [3]:
# hyperparameters
token_size = vocab_size
input_size = 10
hidden_size = 50 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-3

In [4]:
# Utility functions
def ix_to_tensor(ix):
    return Variable(torch.LongTensor([[ix]])).cuda()


def tensor_to_ix(t):
    v,i = torch.max(t,1)
    return i.data[0][0]

In [5]:
# nn module
class CharRNN(nn.Module):
 
    def __init__(self, token_size, input_size, hidden_size):
        super(CharRNN, self).__init__()
        self.token_size = token_size
        self.input_size = input_size
        self.hidden_size = hidden_size

        
        self.encoder = nn.Embedding(token_size, input_size)
        self.rnn = nn.LSTMCell(input_size, hidden_size)
        self.linear = nn.Linear(hidden_size, token_size)
        
    def forward(self, input, h,c):
        emb = self.encoder(input).view(1, -1)
        h,c = self.rnn(emb, (h,c))
        output = self.linear(h)
        return output, h,c

    def init_hidden(self):
        return Variable(torch.FloatTensor(1, self.hidden_size).zero_()).cuda() ,Variable(torch.FloatTensor(1, self.hidden_size).zero_().cuda())

In [ ]:
rnn = CharRNN(token_size, input_size, hidden_size).cuda()
criterion=nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), lr=learning_rate)

def train(inputs, targets, h,c):
    optimizer.zero_grad()

    loss = 0
    for i in range(len(inputs)):
        input = ix_to_tensor(inputs[i])
        target = Variable(torch.LongTensor([targets[i]])).cuda()

        output, h,c = rnn(input, h,c)
        loss += criterion(output, target)
    
    loss.backward()
    optimizer.step()

    return loss.data[0] , h,c

def sample(seed_ix, n):
    h,c = rnn.init_hidden()

    x = ix_to_tensor(seed_ix)
    ixes = []
    for t in range(n):
        output, h,c = rnn(x, h,c)
        ix = tensor_to_ix(output)
        ixes.append(ix)
        x = ix_to_tensor(ix)
        
    return ixes    

In [ ]:
n, p = 0, 0

while True:
    if p+seq_length+1 >= len(data) or n == 0: 
        h,c = rnn.init_hidden()
        p = 0 # go from start of data
        
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    # sample from the model now and then
    if n % 1000 == 0:
        sample_ix = sample( inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print ('----\n {} \n----'.format( txt) )

    # forward seq_length characters through the net and fetch gradient
    loss,h,c = train(inputs, targets, h,c) 
    h = Variable(h.data).cuda()
    c = Variable(c.data).cuda()
    
 
    if n % 1000 == 0:
        print( 'iter {}, loss: {}'.format(n, loss)) # print progress

    p += seq_length # move data pointer
    n += 1 # iteration counter  

----
 什颜，分，，密脱脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从脱从 
----
iter 0, loss: 155.58978271484375
----
 上的，自己己，了的，自己，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的，自己的 
----
iter 1000, loss: 117.32456970214844
----
 来立慢慢了，自己己受伤的么，自己己此的伤的，用了的树枝，用手自己了的一个叶堆，用手手了出的瓶子子
是韩立这个，用手手了出出，自己受伤立慢在么大的树叶堆，用手手自己的一伤的树枝，用手手自己己的伤的，用了，自己己此，韩立在么大的树叶堆，用手手自己的一伤的树枝，用手手自己己的伤的，用了，自己己此，韩立在么大的树叶堆，用手手自己的一伤的树枝，用手手自己己的伤的，用了，自己己此，韩立在么大的树叶堆，用手手自 
----
iter 2000, loss: 91.8636474609375
----
 了铁。
韩立慢慢了枝个枝个，用双手手了瓶子，从有些。
韩立现在了望子个，用手了出，用手自己受此伤的，用子
韩立现在了子个小小瓶子，瓶子是拿瓶子到这个边小瓶子，韩立现在苦难，用手手了出瓶子放子，用手手出出瓶子放子属瓶子表瓶子表面手手了出出来，用手在上胡乱指了什么。
韩立倒吸了，用不有些什么，自己也能也大的树枝，这个月“象甲功”的，自己也道在瞅什么重要，用手了，自己受此伤的，用子
韩立现在了子个小小瓶 
----
iter 3000, loss: 64.01300811767578
----
 指能寄寄